In [ ]:
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path

## Obtain bug prediction scores

In [ ]:
bugs = pd.read_csv(r'C:/Users/migue/Downloads/ruben-demo-code-lifetime-data.csv',  lineterminator='\n')

In [ ]:
bugs.head()

In [ ]:
bugs['bug_fix'].unique()

In [ ]:
groupdf = bugs.groupby(by=["bug_id",'line_number'])

In [ ]:
groupdf.head()

In [ ]:
import math

bugScore = pd.DataFrame()


j = 0
for i in groupdf:
        
    print(i[1].shape)
    i[1]['author_commit_date'] = (i[1]['author_commit_date'] - i[1]['author_commit_date'].min()) / (i[1]['author_commit_date'].max() - i[1]['author_commit_date'].min())
    i[1]['author_commit_date'] = 1/ (1 + math.e**(-12* i[1]['author_commit_date'] + 12))
    
    sumdf = i[1].groupby(by=["bug_fix"]).agg({'author_commit_date' : ['sum']})
    
    if sumdf.size == 1:
        
        twrrevisions = float((sumdf.iloc[0])['author_commit_date'])
        Bc = 0.35 * twrrevisions
        print(Bc)
        defectC = 1 -math.e**(-Bc)
        print(defectC)
    else :
        
        twrRevisions = float((sumdf.iloc[0])['author_commit_date'])
        twrFixRevisions = float((sumdf.iloc[1])['author_commit_date'])
        Bc = 0.35 * twrRevisions + 0.65 * twrFixRevisions
        print(Bc)
        defectC = 1 -math.e**(-Bc)
        print(defectC)
    
    bug_id = int(i[0][0])
    line = int(i[0][1])
    print(bug_id)
    print(line)
    bugScore.at[j,'bug_probability'] = defectC
    bugScore.at[j,'bug_id'] = bug_id
    bugScore.at[j,'line_number'] = line
    bugScore.at[j,'fix_commit_hash'] = i[1].iloc[0]['fix_commit_hash']
    bugScore.at[j,'project_full_name'] = i[1].iloc[0]['project_full_name']
    bugScore.at[j,'bug_type'] = i[1].iloc[0]['bug_type']
    #bugScore.at[j,'file_path'] =  i[0][2]
    
    j= j + 1
 
        
        


In [ ]:
bugScore.head()

In [ ]:
bugScore.shape

In [ ]:
bugScore['bug_probability'].unique()

In [ ]:
bugScore.head(5)

In [ ]:
#bugs.to_csv('..\data\RQ3_Data.csv', index=False, encoding="utf8" )

In [ ]:
bugScore = bugScore.sort_values(by=['bug_probability'],ascending=False)

In [ ]:
length = bugScore.shape[0]
counter = 0
for i in range(length):
    
    bugScore.at[i,'ranking_position']= counter + 1 
    counter = counter + 1

In [ ]:
bugs2 = buggy_elements_df = pd.read_csv('../data/generated/buggy-code-lifetime-data.csv', sep=',')

In [ ]:
bugs2.head()

In [ ]:
bugs2 = bugs2.rename(columns={"buggy_line_number": "line_number"})

In [ ]:
bugs2.shape

In [ ]:
bugs2.dtypes

In [ ]:
bugScore.dtypes

In [ ]:
bugs2['bug_id']=bugs2['bug_id'].astype(str)
bugs2['line_number']=bugs2['line_number'].astype(str)
bugScore['bug_id']=bugScore['bug_id'].astype(str)
bugScore['line_number']=bugScore['line_number'].astype(str)
bugScore['ranking_position']=bugScore['ranking_position'].astype(int)

In [ ]:
def removeZero(x):
    
    return x.split('.')[0]

In [ ]:
bugScore['bug_id'] = bugScore['bug_id'].apply(removeZero)
bugScore['line_number'] = bugScore['line_number'].apply(removeZero)

In [ ]:
final_df = pd.merge(bugScore,bugs2, on=["bug_id",'line_number'], how="inner")

In [ ]:
final_df.head()

In [ ]:
final_df['ranking_position'].unique()

In [ ]:
#top_30_df = buggy_elements.sort_values(by=['Bug Probability'],ascending=False).head(30)
#delete duplicated rows with repeated line,id and file_path ( different buggy code elements)